In [1]:
import rsa_io
import rsa
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
import numpy as np
import h5py
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

In [2]:
# TODO: Load all models programatically like in rsa code
file_name = "/Users/huseyinelmas/Desktop/CCN-Lab/RSA-Models/pixel_wise_motion_model.mat"
pixel_wise_motion_model = rsa_io.load_model(file_name)
pixel_wise_motion_rdm = rsa.create_rdm(pixel_wise_motion_model, metric="correlation", name="pixel_wise_motion")

file_name = "/Users/huseyinelmas/Desktop/CCN-Lab/RSA-Models/avg_pixel_intensity_model.mat"
avg_pixel_intensity_model = rsa_io.load_model(file_name)
avg_pixel_intensity_rdm = rsa.create_rdm(avg_pixel_intensity_model, metric="correlation", name="avg_pixel_intensity")

file_name = "/Users/huseyinelmas/Desktop/CCN-Lab/RSA-Models/total_motion_model.mat"
total_motion_model = rsa_io.load_model(file_name)
total_motion_rdm = rsa.create_rdm(total_motion_model, metric="euclidean", name="total_motion")

path = "/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/RSA_Results/Naive/Still_FF/modelRDMS/flowModel_10_frames_hamming"
flow_rdm = rsa_io.load_rdm(path)

path = "/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/RSA_Results/Naive/Still_FF/modelRDMS/gabor_10_frames_correlation"
gabor_rdm = rsa_io.load_rdm(path)

models = ["constant","total_motion_rdm","pixel_wise_motion_rdm","avg_pixel_intensity_rdm"]

stimuli =["robot-drink","robot-grasp","robot-handwave","robot-talk","robot-nudge",
        "robot-paper","robot-turn","robot-wipe","android-drink","android-grasp","android-handwave",
        "android-talk","android-nudge","android-paper","android-turn","android-wipe","human-drink",
        "human-grasp","human-handwave","human-talk","human-nudge","human-paper","human-turn","human-wipe"]


In [3]:
# Take programmatically
subject_number = 14
experiment_type = "Naive"
stimuli_type = "Still_FF"
electrode_region = "parietal"
path = "/Users/huseyinelmas/Desktop/CCN-Lab/CCN-RSA/RSA_Results/Naive/Still_FF/eegRDMs/"
name = "eeg_rdm_naive_still_parietal_1_correlation"
windowed_eeg_rdm_dict = {}
with h5py.File(path + name + ".hdf5", "r") as f:
    attributes = dict(f.attrs.items())
    for key in list(f.keys()):
        windowed_eeg_rdm_dict[tuple(map(int, key[1:-1].split(',')))] = np.asarray(f[key]["rdm"])


In [4]:
def plot_rdm(rdm,name):
    fig, ax = plt.subplots(figsize=(15,10))
    im = plt.imshow(squareform(rdm))
    plt.title(name)
    ax.set_xticks(np.arange(len(stimuli)))
    ax.set_yticks(np.arange(len(stimuli)))
    # ... and label them with the respective list entries
    ax.set_xticklabels(stimuli)
    ax.set_yticklabels(stimuli)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    fig.colorbar(im)

In [5]:
# Regression part
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
regressor_matrix = np.column_stack((total_motion_rdm,pixel_wise_motion_rdm,avg_pixel_intensity_rdm))
regressor_matrix = sm.add_constant(regressor_matrix)
tvalues_list = []

In [6]:
# VIF calculation
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(regressor_matrix, i) for i in range(regressor_matrix.shape[1])]
vif["features"] = models

In [12]:
type(vif)


pandas.core.frame.DataFrame

In [16]:
vif.to_latex("test")

In [9]:
# Average RDMs for subject
for time_wind, eeg_rdm in windowed_eeg_rdm_dict.items():
    results = OLS(np.mean(eeg_rdm, 0), regressor_matrix).fit()
    for i, model in enumerate(models):
        temp=[experiment_type,stimuli_type,electrode_region,model,time_wind]
        temp.extend([results.params[i],results.tvalues[i],results.pvalues[i]])
        temp.extend(results.conf_int().tolist()[i])
        tvalues_list.append(temp)

In [10]:
column_names = ['experiment_type', 'stimuli_type','electrode_region','model','time_window','beta_coef','tvalues','pvalue','lower_conf_interval','upper_conf_interval']
# TODO: Change name to beta values
tvalues_df = pd.DataFrame(tvalues_list, columns=column_names)
tvalues_df = tvalues_df.sort_values(by="time_window")

In [11]:
tvalues_df

,experiment_type,stimuli_type,electrode_region,model,time_window,beta_coef,tvalues,pvalue,lower_conf_interval,upper_conf_interval
0,Naive,Still_FF,parietal,constant,"(0, 1)",9.223900e-01,17.072760,6.425545e-45,8.160258e-01,1.028754e+00
1,Naive,Still_FF,parietal,total_motion_rdm,"(0, 1)",1.039258e-08,0.088981,9.291621e-01,-2.195442e-07,2.403294e-07
2,Naive,Still_FF,parietal,pixel_wise_motion_rdm,"(0, 1)",-2.115332e-02,-0.287079,7.742706e-01,-1.662182e-01,1.239116e-01
3,Naive,Still_FF,parietal,avg_pixel_intensity_rdm,"(0, 1)",-1.985052e-02,-0.268662,7.883934e-01,-1.653127e-01,1.256117e-01
4,Naive,Still_FF,parietal,constant,"(1, 2)",9.419495e-01,17.286267,1.099938e-45,8.346713e-01,1.049228e+00
5,Naive,Still_FF,parietal,total_motion_rdm,"(1, 2)",1.804385e-08,0.153176,8.783732e-01,-2.138686e-07,2.499563e-07
6,Naive,Still_FF,parietal,pixel_wise_motion_rdm,"(1, 2)",-3.849403e-02,-0.517965,6.049037e-01,-1.848053e-01,1.078172e-01
7,Naive,Still_FF,parietal,avg_pixel_intensity_rdm,"(1, 2)",-1.481385e-02,-0.198787,8.425782e-01,-1.615259e-01,1.318982e-01
448,Naive,Still_FF,parietal,constant,"(2, 3)",9.474008e-01,17.219449,1.910785e-45,8.390831e-01,1.055718e+00
449,Naive,Still_FF,parietal,total_motion_rdm,"(2, 3)",1.731095e-08,0.145544,8.843893e-01,-2.168487e-07,2.514706e-07


In [ ]:
# t-statistics df
stat_list = []

In [ ]:
# T-test
for model_name in beta_columns:
    for time_wind, eeg_rdm in windowed_eeg_rdm_dict.items():
        data = tvalues_df[tvalues_df["time_window"] == time_wind][model_name]
        t_stat, p_val = stats.ttest_1samp(data,0)
        temp=[experiment_type,stimuli_type,electrode_region,model_name,time_wind]
        temp.extend([t_stat, p_val])
        stat_list.append(temp)

In [ ]:
column_names = ['experiment_type', 'stimuli_type','electrode_region','model_name','time_window',"t_stat","p_val"]
stat_df = pd.DataFrame(stat_list, columns=column_names)
stat_df = stat_df.sort_values(by="time_window")



In [ ]:
data

In [ ]:
def plot_experiment_stimuli_comparisons(rdm_df,electrode_region, model_name):
    fig = make_subplots(rows=2, cols=2, shared_yaxes=False,
    subplot_titles=("Naive: Still vs Video", "Prior: Still vs Video", 
                    "Still: Naive vs Prior", "Video: Naive vs Prior"))
    colors = px.colors.qualitative.Plotly 
    
    variables = ['still', 'video']
    constants = ['naive', 'prior']
    conditions = ["naive still", "naive video", "prior still", "prior video"]
    c_no = 0
    
    # Plots for Naive still vs Naive video, Prior still vs Prior video 
    for col_no, constant in enumerate(constants):
        for variable in variables:
            df_to_plot = rdm_df[(rdm_df["model_name"] == model_name) 
                                & (rdm_df["electrode_region"] == electrode_region)
                                & (rdm_df["experiment_type"] == constant) 
                                & (rdm_df["stimulus_type"] == variable)]
            pos_corr = find_significant_rdms(df_to_plot)["time"].tolist()           
            # Plots the kendall tau curve
            fig.add_trace(go.Scatter(x=df_to_plot["time"],
                                    y=df_to_plot["kendall_tau"],
                                    legendgroup=constant + " " + variable,
                                    mode='lines',
                                    name=constant + " " + variable,
                                    line_color = colors[conditions.index(constant + " " + variable)]),
                                    row=1, col=col_no+1)
            
            # Plotting the significance points in below the curves
            fig.add_trace(go.Scatter(
                            mode='markers',
                            legendgroup=constant + " " + variable,
                            name=constant + " " + variable,
                            x=pos_corr,
                            y=[-.1-(c_no*.02) for _ in range(len(pos_corr))],
                            marker=dict( size=2,color= colors[conditions.index(constant + " " + variable)]),
                            showlegend=False),row = 1, col=col_no+1)
            
            fig.update_yaxes(range=[-0.2, 0.6])
            fig.update_yaxes(title_text="Kendall-tau", row=1, col=col_no+1)
            fig.update_xaxes(title_text="Time ms (relative to stimulus onset)",  row=1, col=col_no+1)
            c_no = c_no+1

     # Plots for Prior Still vs Naive Still, Naive video vs Prior video 
    constants = ['still', 'video']
    variables = ['naive','prior']
    c_no = 0
    for col_no, constant in enumerate(constants):
        for  variable in variables:
            
            df_to_plot = rdm_df[(rdm_df["model_name"] == model_name) 
                                & (rdm_df["electrode_region"] == electrode_region)
                                & (rdm_df["experiment_type"] == variable) 
                                & (rdm_df["stimulus_type"] == constant)]
            pos_corr = find_significant_rdms(df_to_plot)["time"].tolist()   
            # Plotting the kendall tau curve
            fig.add_trace(go.Scatter(x=df_to_plot["time"],
                                    y=df_to_plot["kendall_tau"],
                                    mode='lines',
                                    legendgroup=variable + " " + constant,
                                    name= variable + " " + constant,
                                    line_color = colors[conditions.index(variable + " " + constant)]),
                                    row=2, col=col_no+1)
            
            # Significance dots plotting
            fig.add_trace(go.Scatter(mode='markers',
                            legendgroup=variable + " " + constant,
                            name=variable + " " + constant,
                            x=pos_corr,
                            y=[-.1-(c_no*.02) for _ in range(len(pos_corr))],
                            marker=dict(size=2,color= colors[conditions.index(variable + " " + constant)]),
                            showlegend=False),row = 2, col=col_no+1)
            fig.update_yaxes(range=[-0.2, 0.6])
            fig.update_yaxes(title_text="Kendall-tau", row=1, col=col_no+1)
            c_no = c_no+1
    fig.update_layout(height=1000, width=1700,title=electrode_region + " " + model_name,
    xaxis_title="Time ms (relative to stimulus onset)",
    yaxis_title="Kendall-tau",
    font=dict(size=18,color="#000000"))
    plotly.offline.plot(fig, filename = "RSA_Results/new_plots/"+
                        electrode_region + "_" + model_name +'_experiment_stimuli_comparison.html', auto_open=False)    